# 비정형 빅데이터 응용과 실습 - Week 06
외부 웹서비스에서 데이터를 확보하고, 데이터베이스에 저장한 뒤에, SQL 쿼리를 활용해 출력을 해보겠습니다.
____

## 1. Requests 라이브러리를 활용한 데이터 확보
- 다음의 종목 게시판의 글을 불러오겠습니다
- https://finance.daum.net/content/debates/A005930?symbolCode=A005930&page=1&perPage=100&notice=true&pagination=true&totalCount=35146
- 과도한 요청은 서비스에 피해가 줄 수 있으므로, 교육의 목적으로만 아래 코드를 활용해주세요.

In [ ]:
import requests

In [ ]:
#브라우저인척을 하기 위한 헤더 셋팅
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36",
    "referer": "https://finance.daum.net/quotes/A005930?period=day"

}
url = "https://finance.daum.net/content/debates/A005930?symbolCode=A005930&page=1&perPage=100&notice=true&pagination=true&totalCount=35146"
res = requests.get(url,headers=headers)
res.text

## 2. JSON 타입이므로, JSON 라이브러리를 활용해 Parsing

In [ ]:
import json
data = json.loads(res.text)
print(data)

## 3. pprint라이브러리를 활용해 예쁘게 출력

In [ ]:
import pprint
pprint.pprint(data)

In [ ]:
#포스팅 숫자
len(data['data']['posts'])

## 3. 필요한 정보만 추출 후 변수에 저장

In [ ]:
pages = []
for post in data['data']['posts']:

    link = 'https://finance.daum.net/quotes/A005930?period=day#talks/'+str(post['id'])
    print(link)
    print(post['id'])
    print(post['title'])
    print(post['content'])
    page = {
        "link":link,
        "id":post['id'],
        "title":post['title'],
        "content":post['content'],
    }
    pages.append(page)

## 4. MySQL을 활용한 데이터 저장

In [ ]:
import mysql.connector

In [ ]:
from IPython.display import Image
Image(filename='images/database.png') 

### 4-1. Database 생성

In [ ]:

HOST = ""
USER = ""
PASSWORD = ""
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD
)

mycursor = mydb.cursor()
# mycursor.execute("DROP DATABASE mydb")
try:
    mycursor.execute("CREATE DATABASE mydb")
except:
    pass

### 4-2. Table 생성

In [ ]:
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="mydb"
)

mycursor = mydb.cursor()
try:
    mycursor.execute("DROP TABLE posts;")
except:
    pass
mycursor.execute("CREATE TABLE posts (id VARCHAR(255) UNIQUE, content TEXT, title VARCHAR(255), link VARCHAR(255))")


### 4-3. Row 삽입

In [ ]:
sql = "INSERT INTO posts (id, content, title, link) VALUES (%s, %s, %s, %s)"
val = ("1", "안녕하세요","이것은 제목","https://yonsei.ac.kr")
mycursor.execute(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

### 4-4. 삽입된 Row 출력

In [ ]:
sql = "SELECT * FROM posts"
mycursor.execute(sql)
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

### 4-5. 위에서 수집한 데이터 삽입

In [ ]:
for page in pages:
    sql = "INSERT INTO posts (id, content, title, link) VALUES (%s, %s, %s, %s)"
    val = (str(page['id']), page['content'],page['title'],page['link'])
    mycursor.execute(sql, val)

    mydb.commit()

    print(mycursor.rowcount, "record inserted.")

### 4-6. 데이터 확인

In [ ]:
sql = "SELECT * FROM posts"
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

### 4-7. 키워드 기반 검색 쿼리 작성

In [ ]:
keyword = "통신장비"
sql = "SELECT * FROM posts WHERE content LIKE '%{}%'".format(keyword)

mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

### Quiz. 만약 여러 키워드를 포함하고 싶다면?

In [ ]:
keywords = "속보 연기금"


## 5. Connection 끊어주기

In [ ]:
mycursor.close()
mydb.close()